# Problems

In order to use the previous methods we need a discrete environment(virtually non-existent in real world) and we need to know the probabilities of state transitions. 

1. Split up cartpole environment into bins (upper/lower bound of scalar values makes 1 bin)
2. learn the transition with experience replay to estimate them

# Solving Frozen Lake with Value Iteration

we need 3 dictionaries

Rewards: key is [s,a,s'], value is the immediate reward 
<br>
Transitions: key is [s,a], value is a dict of s' and the frequency it occured 
<br>
Values: key is [s], value is the calculated value of being in that state


Steps:
1. Initialise: all values in our value table to zero
2. Explore: 100 random steps, keep track of transitions and rewards
3. Update: For every state s and every action a in this state, calculate the state action values. In the values table, set the value of state s to the max state action value.
4. Test: Run several test episodes to check the average reward of our agent and if it has solved the environment
5. Repeat: Go back to step 2 and repeat the process until the environment has been solved.

In [30]:
import gym
import collections

ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
TEST_EPISODES = 20
EXPLORATION_STEPS = 100
SOLVED = 0.8

In [31]:
env = gym.make(ENV_NAME)

values = collections.defaultdict(float)
transitions = collections.defaultdict(collections.Counter)
rewards = collections.defaultdict(float)

n_actions = env.action_space.n
n_states = env.observation_space.n

epoch = 0
threshold = 0
reward = 0

state = env.reset()

In [32]:
def get_q_value(state, action):
    t_count = transitions[(state,action)]
    t_sum = sum(t_count.values())
    value = 0.0
    
    for target_state, count in t_count.items():
        r = rewards[(state,action, target_state)]
        trans_p = (count/t_sum)
        value += trans_p * (r + GAMMA * values[target_state])
    
    return value

In [33]:
def explore(steps):
    state = env.reset()
    for i in range(steps):
        action = env.action_space.sample()
        new_state, reward, is_done, _ = env.step(action)
        transitions[(state, action)][new_state] += 1
        rewards[(state, action, new_state)] = reward
        state = env.reset() if is_done else new_state

In [34]:
def update():
    for state in range(n_states):
        state_values = [get_q_value(state, action) for action in range(n_actions)]
        #Update the values table with the max of Q value of s,a
        values[state] = max(state_values)

In [35]:
def test(episodes):
    reward = 0
    for e in range(episodes):
        reward += play_episode(env)
        
    return reward/episodes  

In [36]:
def select_action(state):
    best_action, best_value = None, None
    for action in range(env.action_space.n):
        action_value = get_q_value(state, action)
        if best_value is None or best_value < action_value:
            best_value = action_value
            best_action = action

    return best_action

In [37]:
def play_episode( env):
    total_reward = 0.0
    state = env.reset()

    while True:
        action = select_action(state)
        new_state, reward, is_done, _ = env.step(action)
        rewards[(state, action, new_state)] = reward
        transitions[(state, action)][new_state] += 1
        total_reward += reward
        if is_done:
            break
        state = new_state

    return total_reward

In [38]:
if __name__ == "__main__":

    while reward < SOLVED:

        #Explore for 100 steps
        explore(100)

        #Value Iteration
        update()

        # Test X episodes
        reward = test(TEST_EPISODES)

        if reward > threshold:
            print("Epoch: ",epoch," Reward: ", reward)
            threshold = reward

        epoch += 1
        
    print("Solved in %d epochs!" % epoch)

Epoch:  5  Reward:  0.15
Epoch:  7  Reward:  0.35
Epoch:  10  Reward:  0.45
Epoch:  13  Reward:  0.5
Epoch:  14  Reward:  0.65
Epoch:  15  Reward:  0.75
Epoch:  18  Reward:  0.8
Solved in 19 epochs!
